In [13]:
import pandas as pd
import clinicaldg.cxr.Constants as cxrConstants
import clinicaldg.cxr.process as cxrProcess

In [14]:
df_paths = cxrConstants.df_paths
df_paths

{'MIMIC': '/scratch/rc4499/thesis/data/mimic-cxr/clinicaldg/preprocessed.csv',
 'CXP': '/scratch/rc4499/thesis/data/chexpert/clinicaldg/preprocessed.csv',
 'NIH': '/scratch/rc4499/thesis/data/chestxray8/clinicaldg/preprocessed.csv',
 'PAD': '/scratch/rc4499/thesis/data/padchest/clinicaldg/preprocessed.csv'}

In [15]:
def get_frontal_prop(df):
    num_instances = len(df)
    num_pneumonia = df[df["frontal"] == 1]["frontal"].count()
    return num_pneumonia / num_instances

from os.path import exists
def img_exists(path):
    return exists(path)

dfs = {}
for env in cxrConstants.df_paths:
    func = cxrProcess.get_process_func(env)
    df_env = func(pd.read_csv(cxrConstants.df_paths[env]), only_frontal = False)
    # df_env["img_exists"] = df_env["path"].apply(img_exists)
    # df_env = df_env[df_env["img_exists"]]
    print(env, get_frontal_prop(df_env))
    dfs[env] = df_env

MIMIC 0.6452688154893861
CXP 0.8550445342681356
NIH 1.0
PAD 0.690928343045386


In [16]:
all_dfs = pd.concat([
    dfs["MIMIC"],
    dfs["CXP"],
    dfs["NIH"],
    dfs["PAD"]
])

all_dfs = all_dfs.fillna(0)

# Checking For Studies Without Frontals

In [17]:
def has_frontal(frontal_col):
    return frontal_col.any()

with_frontal = all_dfs.groupby("study_id")["frontal"].agg(has_frontal)

In [18]:
with_frontal

study_id
1                      True
10                     True
100                    True
1000                   True
10000                  True
                       ... 
patient64736/study1    True
patient64737/study1    True
patient64738/study1    True
patient64739/study1    True
patient64740/study1    True
Name: frontal, Length: 545012, dtype: bool

In [19]:
indices_without_frontal = with_frontal[~with_frontal].index
indices_without_frontal

Index(['101741800329932598958328421653105387931',
       '102536448805595370148169205121287134564_2',
       '104766698027297958280589609232122055294',
       '105566223538770174232067848507021687050_2',
       '106700336717044272880726422061056904209',
       '107453571276577049847802415416780956245',
       '10781146099800894534157212886940870380',
       '108032392908719484607354620855973001864_2',
       '109719662294069165640529331358364520285',
       '111828104960009130397249758380651308169',
       ...
       'patient07973/study1', 'patient09563/study2', 'patient11412/study3',
       'patient17678/study12', 'patient26100/study1', 'patient33890/study16',
       'patient51623/study6', 'patient53560/study1', 'patient60984/study1',
       'patient63296/study1'],
      dtype='object', name='study_id', length=10661)

In [20]:
images_without_frontal = all_dfs[all_dfs["study_id"].isin(indices_without_frontal)]

In [21]:
images_without_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
44,10001122,/mimic-cxr/p10/p10001122/s53447138/8039752c-2e...,F,60-80,MIMIC,False,53447138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,10001122,/mimic-cxr/p10/p10001122/s53447138/832b57d8-3a...,F,60-80,MIMIC,False,53447138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130,10002013,/mimic-cxr/p10/p10002013/s55312734/227844e4-4c...,F,40-60,MIMIC,False,55312734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131,10002013,/mimic-cxr/p10/p10002013/s55312734/66ccec51-f0...,F,40-60,MIMIC,False,55312734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145,10002221,/mimic-cxr/p10/p10002221/s53781756/8b88f03a-20...,F,60-80,MIMIC,False,53781756,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144208,322226294479918533199030335357419871910,/padchest/216840111366964012819207061112010322...,F,40-60,PAD,False,216840111366964012819207061112010322141113829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144256,274291740032763798884772564498056823907,/padchest/216840111366964012819207061112010281...,F,80-,PAD,False,216840111366964012819207061112010281112908853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144270,120678344150550864453719616733452326527,/padchest/216840111366964012819207061112010307...,M,60-80,PAD,False,216840111366964012819207061112010307095251412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144463,113042320575397495544125056506517027249,/padchest/216840111366964012904401302362010328...,F,60-80,PAD,False,216840111366964012904401302362010328111720589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
images_without_frontal.env.value_counts()

MIMIC    18733
PAD       1030
CXP         19
Name: env, dtype: int64

In [23]:
studies_without_frontal = all_dfs[all_dfs["study_id"].isin(indices_without_frontal)].groupby("env").nunique()["study_id"]
studies_without_frontal

env
CXP        16
MIMIC    9688
PAD       957
Name: study_id, dtype: int64

In [24]:
all_dfs

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Checking Conditional Correlation / Dependence of Frontal / Lateral

If a study has a frontal, whats the proportion of time it also has a lateral...

In [25]:
with_frontal = all_dfs.groupby("study_id")["frontal"].agg(has_frontal)
with_frontal

study_id
1                      True
10                     True
100                    True
1000                   True
10000                  True
                       ... 
patient64736/study1    True
patient64737/study1    True
patient64738/study1    True
patient64739/study1    True
patient64740/study1    True
Name: frontal, Length: 545012, dtype: bool

In [26]:
indices_with_frontal = with_frontal[with_frontal].index
indices_with_frontal

Index(['1', '10', '100', '1000', '10000', '10001',
       '100014625199913409730274754282179594842',
       '100015392272639845228940263464822056020',
       '100016903621660259605956393388617381387', '10002',
       ...
       'patient64731/study1', 'patient64732/study1', 'patient64733/study1',
       'patient64734/study1', 'patient64735/study1', 'patient64736/study1',
       'patient64737/study1', 'patient64738/study1', 'patient64739/study1',
       'patient64740/study1'],
      dtype='object', name='study_id', length=534351)

In [27]:
images_with_frontal = all_dfs[all_dfs["study_id"].isin(indices_with_frontal)]
images_with_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def has_lateral(group):
    return not group.all()

with_lateral = images_with_frontal.groupby("study_id")["frontal"].agg(has_lateral)
with_lateral

study_id
1                      False
10                     False
100                    False
1000                   False
10000                  False
                       ...  
patient64736/study1    False
patient64737/study1    False
patient64738/study1    False
patient64739/study1    False
patient64740/study1    False
Name: frontal, Length: 534351, dtype: bool

In [29]:
indices_with_lateral = with_lateral[with_lateral]
indices_with_lateral

study_id
100014625199913409730274754282179594842    True
100016903621660259605956393388617381387    True
100023894246484279964129120944560776912    True
100035238701184647172015593785663345624    True
100076021040072803090840594086153650249    True
                                           ... 
patient64618/study1                        True
patient64625/study1                        True
patient64640/study1                        True
patient64676/study1                        True
patient64705/study1                        True
Name: frontal, Length: 179545, dtype: bool

In [30]:
with_lateral.value_counts()

False    354806
True     179545
Name: frontal, dtype: int64

In [31]:
images_with_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
images_with_frontal["has_lateral"] = images_with_frontal["study_id"].isin(indices_with_lateral.index)
images_with_frontal["has_lateral"] = images_with_frontal["has_lateral"].map({True: 1, False: 0})
images_with_frontal

<ipython-input-32-996508ebc75a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  images_with_frontal["has_lateral"] = images_with_frontal["study_id"].isin(indices_with_lateral.index)
<ipython-input-32-996508ebc75a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  images_with_frontal["has_lateral"] = images_with_frontal["has_lateral"].map({True: 1, False: 0})


,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema,has_lateral
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [52]:
studies_with_frontal = images_with_frontal.drop_duplicates("study_id")
studies_with_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema,has_lateral
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,10000032,/mimic-cxr/p10/p10000032/s56699142/ea030e7a-2e...,F,40-60,MIMIC,True,56699142,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,10000764,/mimic-cxr/p10/p10000764/s57375967/096052b7-d2...,M,80-,MIMIC,True,57375967,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144488,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,False,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [65]:
studies_with_frontal["has_lateral"].sum()

179545

In [67]:
group_cols = ["env", "has_lateral"]
# group_cols = "env"
disease_cols = ["No Finding", "Atelectasis", "Cardiomegaly", "Effusion", "Pneumonia", "Pneumothorax", "Consolidation", "Edema"]

by_view = studies_with_frontal.groupby(group_cols).sum(disease_cols)[disease_cols]
by_view

No Finding  Atelectasis  Cardiomegaly  Effusion  Pneumonia  \
env   has_lateral                                                               
CXP   0               11335.0      25864.0       19591.0   66749.0     3266.0   
      1                5314.0       3544.0        3474.0    9005.0     1318.0   
MIMIC 0               21513.0      31913.0       26842.0   37155.0     8782.0   
      1               53666.0      12397.0       10589.0   14036.0     6162.0   
NIH   0               21403.0       1665.0         777.0    1240.0      174.0   
PAD   0               21724.0       3025.0        4363.0    3454.0     2122.0   
      1               13788.0       2365.0        4585.0    2458.0     2647.0   

                   Pneumothorax  Consolidation    Edema  
env   has_lateral                                        
CXP   0                 15615.0        11028.0  46504.0  
      1                  1704.0         1734.0   2440.0  
MIMIC 0                  6832.0         8080.0  20631.0  
      1                  2246.0         2088.0   5308.0  
NIH   0                   252.0          436.0     78.0  
PAD   0                   176.0          775.0   1044.0  
      1                   120.0          746.0    143.0

In [68]:
lateral_per_disease = studies_with_frontal[studies_with_frontal["has_lateral"] == 1].groupby("env").sum()[disease_cols]
lateral_per_disease.to_csv("c_cond_lateral.csv")
lateral_per_disease

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,5314.0,3544.0,3474.0,9005.0,1318.0,1704.0,1734.0,2440.0
MIMIC,53666.0,12397.0,10589.0,14036.0,6162.0,2246.0,2088.0,5308.0
PAD,13788.0,2365.0,4585.0,2458.0,2647.0,120.0,746.0,143.0


In [69]:
total_per_disease = studies_with_frontal.groupby("env").sum()[disease_cols]
total_per_disease.to_csv("c_cond_total.csv")
total_per_disease

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,16649.0,29408.0,23065.0,75754.0,4584.0,17319.0,12762.0,48944.0
MIMIC,75179.0,44310.0,37431.0,51191.0,14944.0,9078.0,10168.0,25939.0
NIH,21403.0,1665.0,777.0,1240.0,174.0,252.0,436.0,78.0
PAD,35512.0,5390.0,8948.0,5912.0,4769.0,296.0,1521.0,1187.0


In [70]:
proportion = lateral_per_disease / total_per_disease
proportion.to_csv("c_cond_proportion.csv")
proportion

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,0.319178,0.120511,0.150618,0.118872,0.287522,0.098389,0.135872,0.049853
MIMIC,0.713843,0.279779,0.282894,0.274189,0.412339,0.247411,0.205350,0.204634
NIH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAD,0.388263,0.438776,0.512405,0.415765,0.555043,0.405405,0.490467,0.120472


In [27]:
images_with_lateral = images_with_frontal[images_with_frontal["study_id"].isin(indices_with_lateral.index)]
images_with_lateral

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10000764,/mimic-cxr/p10/p10000764/s57375967/096052b7-d2...,M,80-,MIMIC,True,57375967,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144485,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,True,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144486,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144487,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144488,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,False,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
